In [3]:
#!/usr/bin/env python3

# add the path to the folder above, hardcoded
import sys
pathhere = "/home/luke/mujoco-devel/rl/"
sys.path.insert(0, pathhere)

from env.MjEnv import MjEnv
import numpy as np
from matplotlib import pyplot as plt
import pickle
import scipy.stats
from math import sqrt

# increase matplotlib resolution
plt.rcParams['figure.dpi'] = 600
plt.rcParams['savefig.dpi'] = 600

In [4]:
# get the test data
testdatapath = "/home/luke/gripper-ros/test_data/palm_vs_no_palm"
test_names = "{0}_E{1}_{2}_YCB"
palms = ["no_palm", "palm"]
stiffnesses = [1, 2, 3]
fingers = [45, 60, 75, 90]

# create modelsaver instance
from ModelSaver import ModelSaver
directory = ""#"pb4_tests_apr23" # ""
saver = ModelSaver(testdatapath)

# get test data class
pathtestclass = "/home/luke/gripper-ros/src/rl/gripper_dqn/scripts/"
sys.path.insert(0, pathtestclass)
from grasp_test_data import GraspTestData, set_palm_frc_threshold, set_X_frc_threshold, set_Y_frc_threshold
data_explorer = GraspTestData()
set_palm_frc_threshold(10.0)
set_X_frc_threshold(10.0)
set_Y_frc_threshold(10.0)

# WATCH OUT FOR THIS SETTING
data_explorer.add_palm_start_force = False

# load the results from all of the tests
results = [
[
    [
      [] for f in range(len(fingers))
    ] for s in range(len(stiffnesses))
  ] for p in range(len(palms))
]

binomtests = [
[
    [
      [] for f in range(len(fingers))
    ] for s in range(len(stiffnesses))
  ] for p in range(len(palms))
]

sim_results = [[
  [0.776, 0.868, 0.808, 0.762],
  [0.760, 0.840, 0.800, 0.765],
  [0.784, 0.803, 0.730, 0.669]],
 [[0.826, 0.891, 0.918, 0.910],
  [0.850, 0.903, 0.936, 0.907],
  [0.852, 0.899, 0.900, 0.881]
]]

swap_XY = True
objects_to_swap_XY = [1, 2, 3, 4, 5, 6, 9, 11, 18]

add_palm_force = True

# determine the success rates overall for each object
num_obj_YCB = 18
sr_per_obj = [0 for x in range(num_obj_YCB)]
trials_per_obj = [0 for x in range(num_obj_YCB)]

# measure with/without palm
success_palm_no_palm = [0 for x in range(2)]
trials_palm_no_palm = [0 for x in range(2)]

for i in range(len(palms)):
  for j in range(len(stiffnesses)):
    for k in range(len(fingers)):

      if add_palm_force and palms[i] == "palm":
        data_explorer.add_palm_start_force = True
      else:
        data_explorer.add_palm_start_force = False

      # load this test data
      saver.enter_folder(test_names.format(palms[i], stiffnesses[j], fingers[k]))
      testdata = saver.load(id=None, filenamestarts="test_data", suffix_numbering=True)

      for t in range(len(testdata.trials)):

        num = testdata.trials[t].object_num
        success = testdata.trials[t].stable_height
        sr_per_obj[num - 1] += success
        trials_per_obj[num - 1] += 1

        if testdata.trials[t].object_num in objects_to_swap_XY:
          new_Y = testdata.trials[t].X_frc_tol
          new_X = testdata.trials[t].Y_frc_tol
          testdata.trials[t].X_frc_tol = new_X            
          testdata.trials[t].Y_frc_tol = new_Y

      results[i][j][k] = data_explorer.get_test_results(data=testdata)
      saver.exit_folder()

      # input the simulation results manually
      success = int(results[i][j][k].avg_stable_height * results[i][j][k].num_trials)
      binomtests[i][j][k] = scipy.stats.binomtest(k=success, n=results[i][j][k].num_trials)
      results[i][j][k].sim_SR = sim_results[i][j][k]

      # save extra info
      success_palm_no_palm[i] += success
      trials_palm_no_palm[i] += results[i][j][k].num_trials

Loading file /home/luke/gripper-ros/test_data/palm_vs_no_palm/no_palm_E1_45_YCB/test_data_007.lz4 with pickle ... finished
Loading file /home/luke/gripper-ros/test_data/palm_vs_no_palm/no_palm_E1_60_YCB/test_data_006.lz4 with pickle ... finished
Loading file /home/luke/gripper-ros/test_data/palm_vs_no_palm/no_palm_E1_75_YCB/test_data_005.lz4 with pickle ... finished
Loading file /home/luke/gripper-ros/test_data/palm_vs_no_palm/no_palm_E1_90_YCB/test_data_007.lz4 with pickle ... finished
Loading file /home/luke/gripper-ros/test_data/palm_vs_no_palm/no_palm_E2_45_YCB/test_data_005.lz4 with pickle ... finished
Loading file /home/luke/gripper-ros/test_data/palm_vs_no_palm/no_palm_E2_60_YCB/test_data_003.lz4 with pickle ... finished
Loading file /home/luke/gripper-ros/test_data/palm_vs_no_palm/no_palm_E2_75_YCB/test_data_006.lz4 with pickle ... finished
Loading file /home/luke/gripper-ros/test_data/palm_vs_no_palm/no_palm_E2_90_YCB/test_data_009.lz4 with pickle ... finished
Loading file /ho

In [5]:
header = f"""{"Object num":<10} | {"Trials":<6} | {"SR":<6}"""
line = "{:<10} | {:<6} | {:<6.3f}"
total_trials = 0
total_sr = 0
print(header)
for i in range(len(sr_per_obj)):
  total_trials += trials_per_obj[i]
  total_sr += sr_per_obj[i]
  print(line.format(i + 1, trials_per_obj[i], sr_per_obj[i] / trials_per_obj[i]))
print(line.format("Total", total_trials, total_sr / total_trials))

Object num | Trials | SR    
1          | 108    | 0.583 
2          | 98     | 0.663 
3          | 92     | 0.707 
4          | 107    | 0.533 
5          | 87     | 0.793 
6          | 94     | 0.670 
7          | 127    | 0.402 
8          | 79     | 0.911 
9          | 82     | 0.878 
10         | 83     | 0.867 
11         | 102    | 0.667 
12         | 84     | 0.821 
13         | 99     | 0.687 
14         | 107    | 0.598 
15         | 114    | 0.561 
16         | 114    | 0.482 
17         | 85     | 0.847 
18         | 90     | 0.722 
Total      | 1752   | 0.670 


To determine whether Coin 2 is more biased towards heads than Coin 1 using the results from your coin tossing experiments, you can perform a hypothesis test on the proportions. Here's how you can do it step-by-step:
 
### Step 1: Set Up Your Hypotheses
 
- **Null Hypothesis (H0):** The proportion of heads for Coin 1 is equal to the proportion of heads for Coin 2. In mathematical terms: $ p_1 = p_2 $ or $ p_1 - p_2 = 0 $.
- **Alternative Hypothesis (H1):** The proportion of heads for Coin 2 is greater than the proportion of heads for Coin 1. Thus, $ p_1 < p_2 $ or $ p_1 - p_2 < 0 $.
 
### Step 2: Calculate the Test Statistic
 
The test statistic for comparing two proportions is given by the formula:
 
$$
Z = \frac{(\hat{p}_1 - \hat{p}_2) - 0}{\sqrt{\hat{p}(1-\hat{p})\left(\frac{1}{n_1} + \frac{1}{n_2}\right)}}
$$
 
Where:
- $ \hat{p}_1 $ is the sample proportion from Coin 1 (0.62).
- $ \hat{p}_2 $ is the sample proportion from Coin 2 (0.71).
- $ n_1 $ and $ n_2 $ are the number of trials for Coin 1 and Coin 2 respectively (100 each).
- $ \hat{p} $ is the pooled proportion, calculated since we assume under the null hypothesis that the proportions are equal:
 
$$
\hat{p} = \frac{x_1 + x_2}{n_1 + n_2}
$$
Where $ x_1 = n_1 \times \hat{p}_1 $ and $ x_2 = n_2 \times \hat{p}_2 $.
 
### Step 3: Calculate the Pooled Proportion
 
Using the provided data:
 
$$
\hat{p} = \frac{100 \times 0.62 + 100 \times 0.71}{100 + 100} = \frac{62 + 71}{200} = \frac{133}{200} = 0.665
$$
 
### Step 4: Compute the Test Statistic
 
Plug in the numbers:
 
$$
Z = \frac{(0.62 - 0.71)}{\sqrt{0.665 \times (1 - 0.665) \times (1/100 + 1/100)}} = \frac{-0.09}{\sqrt{0.665 \times 0.335 \times 0.02}}
$$
 
Let's calculate that.
 
### Step 5: Determine the p-value
 
The p-value will tell you the probability of observing a difference as extreme as the one seen (or more) under the null hypothesis. Since this is a one-tailed test (you are testing if one proportion is greater than the other), you'll find the p-value associated with your Z-score.
 
### Step 6: Compare the p-value to Your Significance Level
 
If the p-value is less than 0.05, you reject the null hypothesis, concluding there's a statistically significant difference that Coin 2 is more biased towards heads than Coin 1.
 
Let's perform the calculations for the test statistic and the p-value.
 
### Results of the Calculations
 
- **Pooled Proportion ($\hat{p}$):** 0.665
- **Standard Error:** 0.06675
- **Z-Score:** -1.348
- **P-Value:** 0.0888
 
### Interpretation
 
The Z-score of -1.348 indicates that the observed difference in proportions is 1.348 standard deviations away from the hypothesized mean difference of 0 (under the null hypothesis).
 
The p-value of 0.0888 tells us the probability of observing a difference as extreme as this (or more extreme) under the null hypothesis is about 8.88%. Since this p-value is greater than your significance level ($\alpha = 0.05$), you **do not reject the null hypothesis**.
 
Thus, based on this test, there is not enough evidence to conclude that Coin 2 is more biased towards heads than Coin 1 at the 5% significance level.

In [6]:
def calc_z(success_1, trials_1, success_2, trials_2):
  """
  Calculate the z statistic based on two experiments. The number of successful
  trials (success) and the number of total trials (trials). This assumes a
  bernoulli variable (trials can only be either success, 1, or failure, 0)
  """

  p1 = success_1 / trials_1
  p2 = success_2 / trials_2
  p = (success_1 + success_2) / (trials_1 + trials_2)

  z = (p1 - p2) / (((p*(1-p)/trials_1) + (p*(1-p)/trials_2))**0.5)

  return z

def statistical_significance(test_1, test_2, confidence_level=0.95, printout=True,
                             one_tailed=True):
  """
  Gives significance of test_1 being higher than test_2 = (successes, trials)
  """

  zstat = calc_z(*test_1, *test_2)
  prob = (1 - scipy.stats.norm.cdf(zstat)) * 100
  if not one_tailed: prob *= 2
  if prob < (1 - confidence_level) * 100:
    sig = True
  else: sig = False

  if printout:
    print(f"Significance test gives Z statistic {zstat:.3f}")
    print(f"This corresponds to {'one-tailed' if one_tailed else 'two-tailed'} probability of {prob:.5f} %")
    print(f"Test is significant at {confidence_level * 100:.1f}% confidence level: {sig}")

  return sig, zstat, prob

def find_CI(success, num_trials, confidence_level=0.95):
  """
  Return the confidence interval, assuming that the other test has the same
  number of trials
  """

  test = scipy.stats.binomtest(k=success, n=num_trials)
  return test.proportion_ci(confidence_level=confidence_level)
  

print(success_palm_no_palm)
print(trials_palm_no_palm)

avg_sr = [0.69, 0.82]

z = calc_z(success_palm_no_palm[0], trials_palm_no_palm[0],
           success_palm_no_palm[1], trials_palm_no_palm[1])
print("z is", z)
print("CI is", find_CI(success_palm_no_palm[0], trials_palm_no_palm[0]))
sr_CI_palm = find_CI(int(avg_sr[1] * trials_palm_no_palm[1]), trials_palm_no_palm[1])
pm_CI_palm = 0.5 * (sr_CI_palm.high - sr_CI_palm.low)
pm_CI_palm = max(sr_CI_palm.high - avg_sr[1], avg_sr[1] - sr_CI_palm.low)
sr_CI_no = find_CI(int(avg_sr[0] * trials_palm_no_palm[0]), trials_palm_no_palm[0])
pm_CI_no = 0.5 * (sr_CI_no.high - sr_CI_no.low)
pm_CI_no = max(sr_CI_no.high - avg_sr[0], avg_sr[0] - sr_CI_no.low)
print(f"Palm grasping: {avg_sr[1]:.2f} +-{pm_CI_palm:.3f}")
print(f"No palm grasping: {avg_sr[0]:.2f} +-{pm_CI_no:.3f}")
print("CI strictly", 0.5 * (sr_CI_palm.low + sr_CI_palm.high))
print("CI strictly", 0.5 * (sr_CI_no.low + sr_CI_no.high))

[559, 614]
[929, 823]
z is -6.409724898482009
CI is ConfidenceInterval(low=0.5694224896861767, high=0.6333750045677323)
Palm grasping: 0.82 +-0.029
No palm grasping: 0.69 +-0.031
CI strictly 0.8178014665831824
CI strictly 0.6893841945336523


In [7]:
# do confidence intervals for grocery test, ppo vs heuristic
ppo = [85 + 115, 210]
heuristic = [75 + 94, 210]
MAT = [1]
use_all = [140 + 147 + 136, 450]
no_model = [92 + 95 + 100, 450]
sensor_2 = [119, 150]
sensor_1 = [120, 150]
sensor_0 = [103, 150]

# get confidence interval overlap
# ppo_CI = find_CI(ppo[0], ppo[1], confidence_level=0.95)
# heu_CI = find_CI(heuristic[0], heuristic[1], confidence_level=0.95)
# print("95% confidence intervals:")
# print(f"ppo lower       = {(ppo[0] / ppo[1]) * 100:.2f} - {((ppo[0] / ppo[1]) - ppo_CI.low) * 100:.2f} + {((-ppo[0] / ppo[1]) + ppo_CI.high) * 100:.2f} % (min: {ppo_CI.low * 100:.2f}%)")
# print(f"heuristic upper = {(heuristic[0] / heuristic[1]) * 100:.2f} - {((heuristic[0] / heuristic[1]) - heu_CI.low) * 100:.2f} + {(-heuristic[0] / heuristic[1] + heu_CI.high) * 100:.2f} % (max: {heu_CI.high * 100:.2f}%)")

print("Comparing PPO against heuristic")
statistical_significance(ppo, heuristic)

print("\nComparing using the model to not using the model:")
statistical_significance(use_all, no_model)

print("\nComparing using all sensors to 2 sensors")
statistical_significance(use_all, sensor_2)
print("\nComparing using all sensors to 1 sensors")
statistical_significance(use_all, sensor_1)
print("\nComparing using all sensors to 0 sensors")
statistical_significance(use_all, sensor_0)

print("\nComparing using 2 sensors to 1 sensors")
statistical_significance(sensor_2, sensor_1)
print("\nComparing using 1 sensors to 0 sensors")
statistical_significance(sensor_1, sensor_0)

Comparing PPO against heuristic
Significance test gives Z statistic 4.631
This corresponds to one-tailed probability of 0.00018 %
Test is significant at 95.0% confidence level: True

Comparing using the model to not using the model:
Significance test gives Z statistic 11.108
This corresponds to one-tailed probability of 0.00000 %
Test is significant at 95.0% confidence level: True

Comparing using all sensors to 2 sensors
Significance test gives Z statistic 5.264
This corresponds to one-tailed probability of 0.00001 %
Test is significant at 95.0% confidence level: True

Comparing using all sensors to 1 sensors
Significance test gives Z statistic 5.064
This corresponds to one-tailed probability of 0.00002 %
Test is significant at 95.0% confidence level: True

Comparing using all sensors to 0 sensors
Significance test gives Z statistic 8.172
This corresponds to one-tailed probability of 0.00000 %
Test is significant at 95.0% confidence level: True

Comparing using 2 sensors to 1 sensors


(True, 2.247044921403953, 1.231857818781057)

In [8]:
# print out a table

C = palms
B = stiffnesses
A = fingers

if A == palms and B == stiffnesses:
  a = "use palm"
  b = "stiffness"
  c = "tip angle"
  ijk = "abc"
elif A == palms and B == fingers:
  a = "use palm"
  b = "tip angle"
  c = "stiffness"
  ijk = "acb"
elif A == stiffnesses and B == palms:
  a = "stiffness"
  b = "use palm"
  c = "tip angle"
  ijk = "bac"
elif A == stiffnesses and B == fingers:
  a = "stiffness"
  b = "tip angle"
  c = "use palm"
  ijk = "bca"
elif A == fingers and B == palms:
  a = "tip angle"
  b = "use palm"
  c = "stiffness"
  ijk = "cab"
elif A == fingers and B == stiffnesses:
  a = "tip angle"
  b = "stiffness"
  c = "use_palm"
  ijk = "cba"
else: raise RuntimeError("oopsie")

# variables for tracking confidence intervals
n_CI = len(C) # set collective interval (use 12 for no_palm vs palm)
CI_str = f"{n_CI}x +-CI %"
count_CI = 0
CIs = [None for i in range(n_CI)]
conf_int = 0.95

headings = f"""{a:<10} | {b:<10} | {c:<10} | {"Sim %":<6} | {"Real %":<6} | {"X":<5} | {"Y":<5} | {"Z":<5} | {"numX":<5} | {"numY":<5} | {"numZ":<5} | {"trials":<6} | {"+-CI %":<6} | {CI_str:<10} | {"eff.suc":<8} | {"zstat":<7} | {"p / %":<7}"""
rows = "{0:<10} | {1:<10} | {2:<10} | {3:<6.2f} | {4:<6.2f} | {5:<5.2f} | {6:<5.2f} | {7:<5.2f} | {8:<5} | {9:<5} | {10:<5} | {11:<6} | {12:<6.2f} | {13:<10.2f} | {14:<8} | {15:<7.3f} | {16:<8.2f}"

print(headings)

for i in range(len(A)):
  for j in range(len(B)):
    for k in range(len(C)):

      if ijk == "abc": x = i; y = j; z = k
      elif ijk == "acb": x = i; y = k; z = j
      elif ijk == "bac": x = j; y = i; z = k
      elif ijk == "bca": x = j; y = k; z = i
      elif ijk == "cab": x = k; y = i; z = j
      elif ijk == "cba": x = k; y = j; z = i
      else: raise RuntimeError("oops")

      # calculate the confidence interval
      CI = find_CI(int(results[x][y][z].avg_SR_per_obj * results[x][y][z].num_trials), results[x][y][z].num_trials,
                   confidence_level=conf_int)
      pm_CI = 0.5 * (CI.high - CI.low)

      # determine confidence intervals for groups
      CIs[count_CI] = (int(results[x][y][z].avg_SR_per_obj * results[x][y][z].num_trials), results[x][y][z].num_trials)
      count_CI += 1
      if count_CI == n_CI:
        count_CI = 0
        success = 0
        trials = 0
        for kk in range(n_CI):
          success += CIs[kk][0]
          trials += CIs[kk][1]
        group_CI = find_CI(success, trials, confidence_level=conf_int)
        pm_group_CI = 0.5 * (group_CI.high - group_CI.low) * 100
        if n_CI == 2:
          zstat = calc_z(*CIs[0], *CIs[1])
          prob = (scipy.stats.norm.cdf(zstat)) * 100
      else:
        pm_group_CI = 0.00
        zstat = 0.0
        prob = 0.0

      print(rows.format(
        A[i], B[j], C[k], results[x][y][z].sim_SR * 100, results[x][y][z].avg_SR_per_obj * 100,
        results[x][y][z].avg_X_frc_saturated, results[x][y][z].avg_Y_frc_saturated, results[x][y][z].avg_palm_frc_saturated,
        results[x][y][z].num_X_probe, results[x][y][z].num_Y_probe, results[x][y][z].num_Z_probe, results[x][y][z].num_trials,
        pm_CI * 100, pm_group_CI, round(results[x][y][z].avg_SR_per_obj * results[x][y][z].num_trials),
        zstat, prob
      ))

tip angle  | stiffness  | use_palm   | Sim %  | Real % | X     | Y     | Z     | numX  | numY  | numZ  | trials | +-CI % | 2x +-CI %  | eff.suc  | zstat   | p / %  
45         | 1          | no_palm    | 77.60  | 53.62  | 3.01  | 2.11  | 4.52  | 17    | 15    | 14    | 98     | 10.26  | 0.00       | 53       | 0.000   | 0.00    
45         | 1          | palm       | 82.60  | 60.81  | 2.78  | 2.06  | 6.16  | 16    | 15    | 15    | 83     | 10.96  | 7.45       | 50       | -0.970  | 16.59   
45         | 2          | no_palm    | 76.00  | 54.92  | 2.40  | 1.50  | 4.53  | 18    | 16    | 14    | 95     | 10.39  | 0.00       | 52       | 0.000   | 0.00    


45         | 2          | palm       | 85.00  | 79.52  | 2.90  | 1.76  | 4.24  | 17    | 18    | 17    | 73     | 9.82   | 7.43       | 58       | -3.340  | 0.04    
45         | 3          | no_palm    | 78.40  | 43.81  | 2.91  | 1.42  | 3.16  | 11    | 10    | 7     | 82     | 11.14  | 0.00       | 36       | 0.000   | 0.00    
45         | 3          | palm       | 85.20  | 64.50  | 3.42  | 2.11  | 3.78  | 16    | 17    | 15    | 88     | 10.47  | 7.73       | 57       | -2.737  | 0.31    
60         | 1          | no_palm    | 86.80  | 86.11  | 4.00  | 2.14  | 5.57  | 17    | 17    | 17    | 63     | 9.32   | 0.00       | 54       | 0.000   | 0.00    
60         | 1          | palm       | 89.10  | 88.61  | 3.52  | 2.12  | 6.10  | 18    | 19    | 18    | 64     | 8.80   | 6.28       | 57       | -0.295  | 38.38   
60         | 2          | no_palm    | 84.00  | 90.99  | 3.35  | 1.94  | 5.78  | 18    | 18    | 18    | 63     | 8.00   | 0.00       | 57       | 0.000   | 0.00    
60  

In [14]:
# apply the wilson score to determine confidence intervals
success_rate = 0.67
trials = 45

z = 0.975
p = success_rate
n = trials

ci_max = (1.0 / (1+(z**2 / n))) * ((p + (z**2 / (2*n))) + (z / (2*n)) * sqrt(4*n*p*(1-p) + z**2))
ci_min = (1.0 / (1+(z**2 / n))) * ((p + (z**2 / (2*n))) - (z / (2*n)) * sqrt(4*n*p*(1-p) + z**2))
ci_up = ci_max - p
ci_down = p - ci_min
CI_95 = max(ci_up, ci_down)
CI_95_max = ci_max
CI_95_min = ci_min

print(f"Success rate = {p:.4f}, trials = {n}, confidence bounds are [{ci_min:.4f}, {ci_max:.4f}]")

Success rate = 0.6700, trials = 45, confidence bounds are [0.5988, 0.7342]
